In [2]:
"""
Smoothing utility function: Exponential moving average

- Uses past 30 periods or days to inform prediction (give them more weight) in order to capture trends
- Span corresponds to what is commonly called an “N-day EW moving average”
"""

def smooth(df):
  # cast value column to int for smoothing
  # df.iloc[:, 0] = df.iloc[:, 0].astype(int)
  return df.ewm(span=30.0, ignore_na=True).mean(engine='numba')

In [3]:
# Use and remove seasonality: Use differencing



In [ ]:
"""
Normalizing utility function

Maximum absolute scaling rescales each feature between -1 and 1 by dividing every observation 
by its maximum absolute value
"""
